In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score

In [12]:
X_train = pd.read_csv("valeo_xtrain.csv",header=None,skiprows=[0])
y_train = pd.read_csv("valeo_ytrain.csv").values.ravel()
X_test = pd.read_csv("valeo_xtest.csv",header=None,skiprows=[0])
X_train.shape,y_train.shape,X_test.shape
y_train

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [16]:
X_train['target'] = y_train
df = X_train
df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,target
0,35.7,3.76,49.1,3.78,300.0,1.0,1.0,2.0,111.7,8.0,...,71.52,122.23,20.57,0.55,34.7,2.0,9.54,126.96,11.97,0
1,47.2,3.77,50.3,3.76,30.0,1.0,1.0,2.0,106.0,8.0,...,67.38,163.78,18.73,0.55,38.7,2.0,9.54,133.88,11.97,0
2,52.7,3.78,40.4,3.78,300.0,1.0,1.0,2.0,103.4,8.0,...,89.09,207.73,26.39,0.55,30.2,2.0,9.66,135.28,11.97,0
3,34.9,3.77,34.9,3.78,1000.0,2.0,1.0,1.0,146.0,7.0,...,93.45,177.31,25.73,0.59,17.6,1.0,12.06,116.51,11.97,0
4,50.0,3.77,41.9,3.75,400.0,1.0,1.0,2.0,115.8,8.0,...,85.17,174.73,21.50,0.42,52.0,1.0,12.12,140.92,11.98,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27581,32.9,3.81,39.7,3.77,1000.0,2.0,1.0,1.0,129.4,7.0,...,116.56,201.69,64.99,0.60,18.5,1.0,12.21,122.47,11.97,0
27582,35.9,3.81,36.5,3.79,1300.0,2.0,1.0,1.0,151.8,7.0,...,112.61,196.75,34.91,0.23,14.6,1.0,12.14,122.30,11.97,0
27583,42.9,3.78,44.3,3.77,300.0,1.0,1.0,2.0,89.5,8.0,...,95.33,212.44,30.40,0.55,29.0,2.0,9.61,134.88,11.98,0
27584,44.3,3.75,38.6,3.80,700.0,2.0,1.0,1.0,140.5,7.0,...,65.21,135.55,40.67,0.41,13.8,1.0,12.23,115.81,11.97,0


In [19]:
# Separate majority and minority classes
df_majority = df[df.target==0]
df_minority = df[df.target==1]
df_majority.shape,df_minority.shape

((26893, 28), (693, 28))

In [20]:
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=26893,    # to match majority class
                                 random_state=123) # reproducible results
df_minority_upsampled.shape

(26893, 28)

In [22]:
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.target.value_counts()

1    26893
0    26893
Name: target, dtype: int64

In [23]:
y = df_upsampled.target
X = df_upsampled.drop('target', axis=1)

In [26]:
clf_1 = LogisticRegression().fit(X, y)

E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [30]:
pred_y_1 = clf_1.predict(X)
print( np.unique( pred_y_1 ) )

[0 1]


In [32]:
from sklearn.metrics import accuracy_score
print( accuracy_score(y, pred_y_1) )

0.7009816680920685


In [37]:
clf = GradientBoostingClassifier(criterion='mse', learning_rate=0.01, loss='deviance', max_depth=3, max_features='sqrt', n_estimators=1000)
clf_dec_fct = cross_val_predict(clf, X, y, cv=3, method='decision_function')
roc_auc_score(y, clf_dec_fct)

0.8807586165722265

0.8803532025245142